In [ ]:
# Notebook parameters. Values here are for development only and
# will be overridden when running via snakemake and papermill.

config_file = "../../../config/agam.yaml"

# Finalize cohorts

Here we finalise the list of cohorts to include in the site, removing cohorts which fail H12 window size calibration.

In [ ]:
import yaml
import pandas as pd
from selection_atlas.setup import AtlasSetup

# Initialise the atlas setup.
setup = AtlasSetup(config_file)

In [ ]:
df_cohorts = pd.read_csv(setup.cohorts_file)
df_cohorts.head()

In [ ]:
window_size = []

for cohort_id in df_cohorts.loc[:, "cohort_id"]:
    # Load window sizes.
    h12_calibration_file = setup.h12_calibration_files.as_posix().format(
        cohort=cohort_id
    )
    with open(h12_calibration_file) as params_file:
        window_size_params = yaml.safe_load(params_file)

    window_size.append(window_size_params["h12_window_size"])

df_cohorts["h12_window_size"] = window_size

In [ ]:
df_cohorts = df_cohorts.query("~h12_window_size.isna()", engine="python")
df_cohorts.to_csv(setup.final_cohorts_file, index=None)